In [1]:
#needed imports from Pandas, Numpy, Pickle, Matplot, Keras
import pandas as pd
import numpy as np
import pickle
from pprint import pprint
from os import listdir
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


In [2]:
#loading raw data
path = 'MovementAAL/dataset/MovementAAL_RSS_'
sequences = list()
for i in range (1,120):
    file_path = path + str(i) + '.csv'
    print(file_path)
    df = pd.read_csv(file_path, header=0)
    values = df.values
    sequences.append(values)

MovementAAL/dataset/MovementAAL_RSS_1.csv
MovementAAL/dataset/MovementAAL_RSS_2.csv
MovementAAL/dataset/MovementAAL_RSS_3.csv
MovementAAL/dataset/MovementAAL_RSS_4.csv
MovementAAL/dataset/MovementAAL_RSS_5.csv
MovementAAL/dataset/MovementAAL_RSS_6.csv
MovementAAL/dataset/MovementAAL_RSS_7.csv
MovementAAL/dataset/MovementAAL_RSS_8.csv
MovementAAL/dataset/MovementAAL_RSS_9.csv
MovementAAL/dataset/MovementAAL_RSS_10.csv
MovementAAL/dataset/MovementAAL_RSS_11.csv
MovementAAL/dataset/MovementAAL_RSS_12.csv
MovementAAL/dataset/MovementAAL_RSS_13.csv
MovementAAL/dataset/MovementAAL_RSS_14.csv
MovementAAL/dataset/MovementAAL_RSS_15.csv
MovementAAL/dataset/MovementAAL_RSS_16.csv
MovementAAL/dataset/MovementAAL_RSS_17.csv
MovementAAL/dataset/MovementAAL_RSS_18.csv
MovementAAL/dataset/MovementAAL_RSS_19.csv
MovementAAL/dataset/MovementAAL_RSS_20.csv
MovementAAL/dataset/MovementAAL_RSS_21.csv
MovementAAL/dataset/MovementAAL_RSS_22.csv
MovementAAL/dataset/MovementAAL_RSS_23.csv
MovementAAL/dataset/

In [3]:
#target values
targets = pd.read_csv('MovementAAL/dataset/MovementAAL_target.csv')
targets = targets.values[:,1]
print(targets)

[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]


In [4]:
#Loading grouping
groups = pd.read_csv('MovementAAL/groups/MovementAAL_DatasetGroup.csv', header=0)
groups = groups.values[:,1]
print(groups)

[1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 1 1 1 1 1 1 1
 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 1 1 1 1 1 1 1 2 2 2 2 2 2 2 3
 3 3 3 3 3 3 1 1 1 1 1 1 1 2 2 2 2 2 2 3 3 3 3 3 3 3 1 2 3 4 1 1 2 3 4 2 1
 2 3 4 3 1 2 3 4 4]


In [5]:
len_sequences = []
for one_seq in sequences:
    len_sequences.append(len(one_seq))
print(pd.Series(len_sequences).describe())

count    119.000000
mean      59.067227
std       24.866331
min       30.000000
25%       30.000000
50%       63.000000
75%       72.500000
max      119.000000
dtype: float64


In [6]:
#Padding the sequence with the values in last row to max length
to_pad = 119
new_seq = []
for one_seq in sequences:
    len_one_seq = len(one_seq)
    last_val = one_seq[-1]
    n = to_pad - len_one_seq
   
    to_concat = np.repeat(one_seq[-1], n).reshape(4, n).transpose()
    new_one_seq = np.concatenate([one_seq, to_concat])
    new_seq.append(new_one_seq)
final_seq = np.stack(new_seq)
pprint(new_seq)

[array([[ -2,   1,   3,   3],
       [  0,   1,   4,   3],
       [  0,   1,   4,   3],
       [  0,   2,   5,   6],
       [  2,   2,   6,   6],
       [  2,   2,   6,   6],
       [  2,   4,   8,  16],
       [  2,   6,  11,  16],
       [  4,   6,  11,  16],
       [  4,  10,  15,  21],
       [  7,  10,  16,  26],
       [  7,  10,  16,  26],
       [  9,  11,  16,  31],
       [  9,  11,  17,  31],
       [  9,  11,  17,  31],
       [  9,  17,  17,  36],
       [  9,  17,  20,  40],
       [ 10,  17,  20,  40],
       [ 10,  17,  21,  41],
       [ 12,  19,  26,  41],
       [ 13,  19,  26,  42],
       [ 13,  20,  30,  54],
       [ 14,  19,  29,  54],
       [ 14,  19,  29,  54],
       [ 14,  19,  29,  62],
       [ 15,  26,  30,  62],
       [ 15,  26,  30,  66],
       [ 15,  26,  32,  74],
       [ 15,  26,  24,  76],
       [ 15,  26,  27,  76],
       [ 15,  14,  27,  81],
       [ 20,  14,  33,  90],
       [ 20,  14,  34,  99],
       [ 20,  16,  35, 100],
       [ 20, 

In [7]:
#truncate the sequence to length 60
seq_len = 75
final_seq=sequence.pad_sequences(final_seq, maxlen=seq_len, padding='post', dtype='float', truncating='post')
pprint(final_seq)

array([[[-2.,  1.,  3.,  3.],
        [ 0.,  1.,  4.,  3.],
        [ 0.,  1.,  4.,  3.],
        ...,
        [ 0.,  1.,  6., 10.],
        [ 0.,  1.,  6., 10.],
        [ 0.,  1.,  6., 10.]],

       [[ 4., -2.,  0.,  3.],
        [ 4., -2.,  0.,  3.],
        [-6., -4.,  3.,  2.],
        ...,
        [ 0., -1.,  3.,  7.],
        [ 0., -1.,  3.,  7.],
        [ 0., -1.,  3.,  7.]],

       [[ 0., -2.,  3.,  5.],
        [ 0., -4.,  3.,  3.],
        [-1., -5.,  3.,  3.],
        ...,
        [ 1., -5.,  7.,  5.],
        [ 1., -5.,  7.,  5.],
        [ 1., -5.,  7.,  5.]],

       ...,

       [[-2.,  1.,  3.,  3.],
        [ 0.,  1.,  4.,  3.],
        [ 0.,  1.,  4.,  3.],
        ...,
        [ 0.,  1.,  4.,  3.],
        [ 0.,  1.,  4.,  3.],
        [ 0.,  1.,  4.,  3.]],

       [[-2.,  1.,  3.,  3.],
        [ 0.,  1.,  4.,  3.],
        [ 0.,  1.,  4.,  3.],
        ...,
        [ 0.,  1.,  4.,  3.],
        [ 0.,  1.,  4.,  3.],
        [ 0.,  1.,  4.,  3.]],

       [[-2.

In [8]:
#Training data based on group 2
train = [final_seq[i] for i in range(len(groups)) if (groups[i]==2)]
#validation data based on group 1
validation = [final_seq[i] for i in range(len(groups)) if groups[i]==1]
#test data based on group 3
test = [final_seq[i] for i in range(len(groups)) if groups[i]==3]

#train target based on group 2
train_target = [targets[i] for i in range(len(groups)) if (groups[i]==2)]
#validation target based on group 1
validation_target = [targets[i] for i in range(len(groups)) if groups[i]==1]
#test target based on group 3
test_target = [targets[i] for i in range(len(groups)) if groups[i]==3]

pprint(train)

[array([[  8.,   6.,  17.,  13.],
       [  8.,   4.,  17.,  12.],
       [  7.,   4.,  19.,  12.],
       [  7.,   4.,  19.,  12.],
       [  6.,   9.,  19.,  14.],
       [  6.,   9.,  19.,  14.],
       [  6.,   9.,  19.,  14.],
       [  8.,   7.,  23.,  16.],
       [  8.,   7.,  22.,  16.],
       [  8.,   7.,  22.,  16.],
       [  8.,   5.,  22.,  16.],
       [  7.,   5.,  22.,  16.],
       [  7.,   5.,  22.,  16.],
       [  7.,  10.,  21.,  18.],
       [ 10.,  10.,  21.,  19.],
       [ 10.,  10.,  21.,  19.],
       [ 10.,  13.,  22.,  23.],
       [ 10.,  13.,  22.,  26.],
       [ 10.,  13.,  22.,  26.],
       [ 10.,  17.,  25.,  26.],
       [ 16.,  17.,  25.,  37.],
       [ 16.,  17.,  25.,  37.],
       [ 16.,  15.,  30.,  37.],
       [ 18.,  15.,  31.,  42.],
       [ 18.,  15.,  31.,  42.],
       [ 18.,  16.,  29.,  47.],
       [ 18.,  17.,  29.,  53.],
       [ 18.,  17.,  29.,  53.],
       [ 18.,  20.,  29.,  58.],
       [ 20.,  20.,  31.,  63.],
       [ 

In [9]:
#creating np.arrays for each dataset
train = np.array(train)
validation = np.array(validation)
test = np.array(test)

#training target data
train_target = np.array(train_target)
train_target = (train_target+1)/2

#validation target
validation_target = np.array(validation_target)
validation_target = (validation_target+1)/2

#test data and target test
test_target = np.array(test_target)
test_target = (test_target+1)/2

In [10]:
#adding the LSTM to the model and printing the summary
model = Sequential()
model.add(LSTM(120, input_shape=(seq_len, 4)))
model.add(Dense(1, activation='sigmoid'))
model.summary()
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 120)               60000     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 121       
Total params: 60,121
Trainable params: 60,121
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 120)               60000     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 121       
Total params: 60,121
Trainable params: 60,121
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
adam = Adam(lr=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(train, train_target, epochs=200, batch_size=120, callbacks=[chk], validation_data=(validation,validation_target))


Train on 38 samples, validate on 38 samples
Epoch 1/200


In [12]:
#loading the exported pkl model and testing the accuracy score
model = load_model('best_model.pkl')
from sklearn.metrics import accuracy_score

OSError: Unable to open file (file read failed: time = Sat Jan 23 09:31:29 2021
, filename = 'best_model.pkl', file descriptor = 72, errno = 21, error message = 'Is a directory', buf = 0x7ffee93dd878, total read size = 8, bytes this sub-read = 8, bytes actually read = 18446744073709551615, offset = 0)